# Generate a coorindate / pixel list
This notebook will genereate three lists. This lists are used by server to calculate the pixel in the forecast image with longitude and latitude. 
### listLatitudeComplete
All latitude values in one list 
### listLongitudeComplete
All longitude values in one list 
### listCoordinates
For each pair of latitude / longitude the pixel 

Example: The 5th element in the latitude and longitude together list will give you something like [latitude, longitude]. The 5th element in listCoordinates will give you the pixel in the forecast of a 900x900 matrix. It will look like [y, x].

In [1]:
import wradlib as wrl
import matplotlib.pyplot as pl
import numpy as np
import pandas as pd
import pickle

In [2]:
radolan_grid_ll = wrl.georef.get_radolan_grid(1100,900, wgs84=True)
# The edges of the grid 
#corner up, right
print(radolan_grid_ll[1099,899, :])
#corner down, left
print(radolan_grid_ll[0, 0, :])
#corner up, left
print(radolan_grid_ll[899, 0, :])
#corner down, right
print(radolan_grid_ll[0,899, :])

[17.09545175 55.52653305]
[ 4.67593418 46.19287861]
[ 3.4456198  53.80105162]
[15.46817524 46.18345478]


In [3]:
radolan_grid_ll = wrl.georef.get_radolan_grid(900,900, wgs84=True) # [höhe (latitude), breite (longitude)]

listCoordinates = []
listLatitudeComplete = [] # all latitude values for each box in the grid
listLongitudeComplete = [] # all longitude values for each box in the grid
for hoehe in range(899):
    for breite in range(899):
        listLatitudeComplete.append(radolan_grid_ll[hoehe, breite, :][1])
        listLongitudeComplete.append(radolan_grid_ll[hoehe, breite, :][0])
        listCoordinates.append([hoehe, breite])
        

f = open('listLatitudeComplete.pckl', 'wb')
pickle.dump(listLatitudeComplete, f)
f.close()

f = open('listLongitudeComplete.pckl', 'wb')
pickle.dump(listLongitudeComplete, f)
f.close()

f = open('listCoordinates.pckl', 'wb')
pickle.dump(listCoordinates, f)
f.close()

In [6]:
listCoordinates

[[0, 0],
 [0, 1],
 [0, 2],
 [0, 3],
 [0, 4],
 [0, 5],
 [0, 6],
 [0, 7],
 [0, 8],
 [0, 9],
 [0, 10],
 [0, 11],
 [0, 12],
 [0, 13],
 [0, 14],
 [0, 15],
 [0, 16],
 [0, 17],
 [0, 18],
 [0, 19],
 [0, 20],
 [0, 21],
 [0, 22],
 [0, 23],
 [0, 24],
 [0, 25],
 [0, 26],
 [0, 27],
 [0, 28],
 [0, 29],
 [0, 30],
 [0, 31],
 [0, 32],
 [0, 33],
 [0, 34],
 [0, 35],
 [0, 36],
 [0, 37],
 [0, 38],
 [0, 39],
 [0, 40],
 [0, 41],
 [0, 42],
 [0, 43],
 [0, 44],
 [0, 45],
 [0, 46],
 [0, 47],
 [0, 48],
 [0, 49],
 [0, 50],
 [0, 51],
 [0, 52],
 [0, 53],
 [0, 54],
 [0, 55],
 [0, 56],
 [0, 57],
 [0, 58],
 [0, 59],
 [0, 60],
 [0, 61],
 [0, 62],
 [0, 63],
 [0, 64],
 [0, 65],
 [0, 66],
 [0, 67],
 [0, 68],
 [0, 69],
 [0, 70],
 [0, 71],
 [0, 72],
 [0, 73],
 [0, 74],
 [0, 75],
 [0, 76],
 [0, 77],
 [0, 78],
 [0, 79],
 [0, 80],
 [0, 81],
 [0, 82],
 [0, 83],
 [0, 84],
 [0, 85],
 [0, 86],
 [0, 87],
 [0, 88],
 [0, 89],
 [0, 90],
 [0, 91],
 [0, 92],
 [0, 93],
 [0, 94],
 [0, 95],
 [0, 96],
 [0, 97],
 [0, 98],
 [0, 99],
 [0, 100],

In [4]:
import json
with open('listLatitudeComplete.json', 'w') as f:
    json.dump(listLatitudeComplete, f)

with open('listLongitudeComplete.json', 'w') as f:
    json.dump(listLongitudeComplete, f)

with open('listCoordinates.json', 'w') as f:
    json.dump(listCoordinates, f)

In [11]:
print(max(listLatitudeComplete))
print(max(listLongitudeComplete))

54.88782802327843
15.687563195908872


In [14]:
height = 56
width = 457
coordinate = 0; 
for i in range(len(listCoordinates)):
    if(listCoordinates[i][0] == height and listCoordinates[i][1] == width):
        coordinate = [listLatitudeComplete[i], listLongitudeComplete[i]]
        print(coordinate)

[47.66206974368475, 9.172706726253896]
